In [1]:
# xmlrpc_server.ipynb

import xmlrpc.server as server
import xmlrpc.client as client
import datetime
import pandas as pd
import pickle
import numpy


class RequestHandler(server.SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = server.SimpleXMLRPCServer(("localhost", 8008),
                            requestHandler=RequestHandler)
server.register_introspection_functions()
stats_server = client.ServerProxy("http://localhost:8018")

# Добавление в лог через сервер
def add_log(log_line):
    try:
        stats_server.add_log(log_line)
        print('loged')
        return True
    finally:
        print('NOT loged')
        return False

def get_logs_slice(p_type, s_time, e_time):
    try:
        return stats_server.get_logs_slice(p_type, s_time, e_time)
    except:
        return 'Log server is not available'
server.register_function(get_logs_slice, 'get_logs_slice')

# Тест
def ping():
    add_log('ping')
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    add_log('now')
    return datetime.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
    add_log('type')
    return str(arg), str(type(arg)), arg
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
    add_log('sum')
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
    add_log('pow')
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(sname):
    add_log('black_list_check')
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any(frame['Surname'] == sname)
    if exist:
        return sname + ": "+ "bad_boy"
    else:
        return sname + ": "+ "good_boy"
server.register_function(black_list_check, 'black_list_check')

# Бинарная передача данных
def send_back_binary(bin_data):
    add_log('send_back_binary')
    data = bin_data.data
    return client.Binary(data)
server.register_function(send_back_binary, 'send_back_binary')

# Задание 4. Проверка на наличие человека в черном списке с проверкой по ФИО и дате рождения
def black_list_check_by_name_and_birthday(sname, name, pat, b_date):
    add_log('black_list_check_by_name_and_birthday')
    # Чтение данных из CSV файла
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    
    # Фильтруем DataFrame, проверяя каждую колонку на совпадение
    exist = frame[
        (frame['Surname'] == sname) & 
        (frame['Name'] == name) & 
        (frame['Patronym'] == pat) & 
        (frame['Birth'] == b_date)
    ]
    
    # Проверяем, если есть совпадения
    if not exist.empty:
        return f"{sname} {name} {pat} {b_date}: bad_boy"
    else:
        return f"{sname} {name} {pat} {b_date}: good_boy"
server.register_function(black_list_check_by_name_and_birthday, 'black_list_check_by_name_and_birthday')

# Задание 5. Модифицировать функцию сервера send_back_inversion на корректную работу также с монохромными изображениями,
# т.е. входными данными могут быть размерности (M, N, 3) и (M, N, 1).
def send_back_inversion(bin_data):
    add_log('send_back_inversion')
    img_arr = pickle.loads(bin_data.data)
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    for i in range(height):
        for j in range(width):
            if type(img_arr[i][j]) == numpy.ndarray:
                for p in range(3):
                    img_arr[i][j][p] = 255 - img_arr[i][j][p]
            else:
                img_arr[i][j] = 255 - img_arr[i][j]
    pimg = pickle.dumps(img_arr)
    return client.Binary(pimg)
server.register_function(send_back_inversion, 'color_inversion')

# Задание 6. Функция бинаризации изображения по задаваемому порогу 1-255 для цветных и монохромных изображений
def binarize_by_threshold(bin_data, threshold):
    add_log('binarize_by_threshold')
    img_arr = pickle.loads(bin_data.data)
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    for i in range(height):
        for j in range(width):
            if type(img_arr[i][j]) == numpy.ndarray:
                for p in range(3):
                    img_arr[i][j][p] = 255 if img_arr[i][j][p] >= threshold else 0
            else:
                img_arr[i][j] = 255 if img_arr[i][j] >= threshold else 0
    pimg = pickle.dumps(img_arr)
    return client.Binary(pimg)
server.register_function(binarize_by_threshold, 'binarize_by_threshold')

# Задание 7. Функция разворота изображения относительно вертикали для цветных и монохромных изображений
def mirror_vertically(bin_data):
    add_log('mirror_vertically')
    img_arr = pickle.loads(bin_data.data)
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    for i in range(height):
        for j in range(width // 2):
            if type(img_arr[i][j]) == numpy.ndarray:
                t1, t2 = [p for p in img_arr[i][j]], [p for p in img_arr[i][-j - 1]]
                img_arr[i][j], img_arr[i][-j - 1] = t2, t1
            else:
                img_arr[i][j], img_arr[i][-j - 1] = img_arr[i][-j - 1], img_arr[i][j]
    pimg = pickle.dumps(img_arr)
    return client.Binary(pimg)
server.register_function(mirror_vertically, 'mirror_vertically')


print("Listening on port 8008...")
server.serve_forever()

Listening on port 8008...
loged
NOT loged


127.0.0.1 - - [19/Dec/2024 14:22:33] "POST /RPC2 HTTP/1.1" 200 -


loged
NOT loged


127.0.0.1 - - [19/Dec/2024 14:22:37] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2024 14:25:15] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2024 14:25:19] "POST /RPC2 HTTP/1.1" 200 -


loged
NOT loged


...

loged
NOT loged


127.0.0.1 - - [19/Dec/2024 14:32:21] "POST /RPC2 HTTP/1.1" 200 -


loged
NOT loged


127.0.0.1 - - [19/Dec/2024 14:32:47] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2024 14:33:11] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2024 14:33:15] "POST /RPC2 HTTP/1.1" 200 -


KeyboardInterrupt: 